For this Task we are going to fine tune a pretrained model and deploy it

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint ,ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from PIL import Image

# Load Data 

In [2]:
Augmented_data = tf.keras.utils.image_dataset_from_directory(r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset\Training')
test_data = tf.keras.utils.image_dataset_from_directory(r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset/Testing')
validation_data = tf.keras.utils.image_dataset_from_directory(r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset/Validation')

Found 3087 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.


In [3]:
train_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset\Training', target_size=(224,224), batch_size=10)
test_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset/Testing', target_size=(224,224), batch_size=10)
validation_batch = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=r'C:\Users\hazem\OneDrive\Desktop\Computer vision\Week-1\Teeth_Dataset/Validation', target_size=(224,224), batch_size=10)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [4]:
img, labels = next(train_batch)

In [7]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [9]:
vgg16_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
type(vgg16_model)

keras.src.models.functional.Functional

Take all model Layers expect last layer 

In [8]:
#import keras
from tensorflow import keras

model  = keras.Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [9]:
for layer in model.layers:
    layer.trainable = False

In [10]:
model.add(Dense(units=7, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │        28,679 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,289,223 (512.27 MB)

 Trainable params: 28,679 (112.03 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [11]:
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)

batch_size = 10
train_steps= len(train_batch) // batch_size
validation_steps = len(validation_batch) // batch_size

model.fit(
      x=train_batch, 
      validation_data=validation_batch,
      steps_per_epoch=train_steps,
      validation_steps=validation_steps,
      epochs=20,
      verbose=2,
      callbacks=[early_stopping, reduce_lr]
  )

Epoch 1/20


c:\Users\hazem\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 - 48s - 2s/step - accuracy: 0.4000 - loss: 1.8300 - val_accuracy: 0.6200 - val_loss: 1.2841 - learning_rate: 0.0010
Epoch 2/20
30/30 - 45s - 2s/step - accuracy: 0.5623 - loss: 1.3492 - val_accuracy: 0.5600 - val_loss: 1.4708 - learning_rate: 0.0010
Epoch 3/20
30/30 - 44s - 1s/step - accuracy: 0.6433 - loss: 1.1723 - val_accuracy: 0.6700 - val_loss: 0.8816 - learning_rate: 0.0010
Epoch 4/20
30/30 - 44s - 1s/step - accuracy: 0.6867 - loss: 0.9819 - val_accuracy: 0.5600 - val_loss: 1.1910 - learning_rate: 0.0010
Epoch 5/20
30/30 - 43s - 1s/step - accuracy: 0.7367 - loss: 0.8437 - val_accuracy: 0.7600 - val_loss: 0.7186 - learning_rate: 0.0010
Epoch 6/20
30/30 - 45s - 1s/step - accuracy: 0.7267 - loss: 0.7659 - val_accuracy: 0.6800 - val_loss: 0.8200 - learning_rate: 0.0010
Epoch 7/20
30/30 - 45s - 1s/step - accuracy: 0.8167 - loss: 0.5842 - val_accuracy: 0.7700 - val_loss: 0.6464 - learning_rate: 0.0010
Epoch 8/20
30/30 - 45s - 2s/step - accuracy: 0.7933 - loss: 0.5455 - val_accurac

c:\Users\hazem\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


30/30 - 13s - 445ms/step - accuracy: 0.7778 - loss: 0.5209 - val_accuracy: 0.6786 - val_loss: 0.6253 - learning_rate: 0.0010
Epoch 12/20
30/30 - 46s - 2s/step - accuracy: 0.8567 - loss: 0.4184 - val_accuracy: 0.7900 - val_loss: 0.5657 - learning_rate: 0.0010
Epoch 13/20
30/30 - 46s - 2s/step - accuracy: 0.8467 - loss: 0.4318 - val_accuracy: 0.7900 - val_loss: 0.6039 - learning_rate: 0.0010
Epoch 14/20
30/30 - 45s - 2s/step - accuracy: 0.8822 - loss: 0.3304 - val_accuracy: 0.7700 - val_loss: 0.5809 - learning_rate: 0.0010
Epoch 15/20
30/30 - 45s - 2s/step - accuracy: 0.8933 - loss: 0.3000 - val_accuracy: 0.8400 - val_loss: 0.4064 - learning_rate: 2.0000e-04
Epoch 16/20
30/30 - 46s - 2s/step - accuracy: 0.9100 - loss: 0.2996 - val_accuracy: 0.8500 - val_loss: 0.4059 - learning_rate: 2.0000e-04
Epoch 17/20
30/30 - 45s - 2s/step - accuracy: 0.9133 - loss: 0.2807 - val_accuracy: 0.9200 - val_loss: 0.3545 - learning_rate: 2.0000e-04
Epoch 18/20
30/30 - 45s - 1s/step - accuracy: 0.9100 - loss

In [14]:
model.evaluate(test_batch)

c:\Users\hazem\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 57/103 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.8649 - loss: 0.3852

In [5]:
model.evaluate(test_batch)

c:\Users\hazem\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


103/103 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8526 - loss: 0.4197


[0.40526536107063293, 0.8618677258491516]

In [13]:
model.save('vgg16_model.h5')